In [ ]:
import minfig

minnie = minfig.configure_minnie(return_virtual_module=True)

In [ ]:
import minfig
minnie = minfig.configure_minnie(return_virtual_module=True)

my_mesh = minnie.Decimation.head(1)["mesh"][0]
print(my_mesh.segment_id)

In [ ]:
result = (minnie.Decimation & "segment_id=72347797059274433").fetch1("mesh")
result.vertices

In [ ]:
result.faces

In [ ]:
# Testing 
import datajoint as dj
m65 = dj.create_virtual_module('m65', 'microns_minnie65_01')
schema = dj.schema("microns_minnie65_01")

In [ ]:
m65.WholeAutoAnnotationsLabelClusters3()

In [ ]:
m65.ProofreadLabel()

In [ ]:
m65.LabelKey()

# Building Table Methods

In [108]:
import datajoint as dj
import numpy as np
m65 = dj.create_virtual_module('m65', 'microns_minnie65_01')
schema = dj.schema("microns_minnie65_01")

/usr/local/lib/python3.6/dist-packages/datajoint/connection.py:215: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


# access to the decimated mesh store

In [109]:
import minfig
minnie = minfig.configure_minnie(return_virtual_module=True)
neuron_mesh = (minnie.Decimation & "segment_id=104726695973782750").fetch1("mesh")
len(neuron_mesh.vertices),len(neuron_mesh.faces)

(177972, 355974)

In [99]:
search_dict = dict(segment_id = 103178515351946567)
(m65.FromNeuromancer & search_dict).fetch()

array([(103178515351946567, 0, 'LEAH', 6472723, 12361728, array([[1002207.875  ,  486741.25   ,  658210.8125 ],
       [1002161.75   ,  486676.40625,  658262.8125 ],
       [1002097.3125 ,  486763.0625 ,  658212.9375 ],
       ...,
       [1436672.     ,  868272.     ,  628120.     ],
       [1436672.     ,  869632.     ,  627380.     ],
       [1436672.     ,  869776.     ,  627320.     ]]), array([[      0,       2,       1],
       [      0,       1,       3],
       [      0,       3,       4],
       ...,
       [6470561, 6470559, 6471377],
       [6470635, 6470634, 6471460],
       [6470911, 6470909, 6470916]], dtype=uint32)),
       (103178515351946567, 1, 'LEAH', 6472723, 12361728, array([[1002207.875  ,  486741.25   ,  658210.8125 ],
       [1002161.75   ,  486676.40625,  658262.8125 ],
       [1002097.3125 ,  486763.0625 ,  658212.9375 ],
       ...,
       [1436672.     ,  868272.     ,  628120.     ],
       [1436672.     ,  869632.     ,  627380.     ],
       [1436672.   

In [100]:
ids,names,version = m65.FromNeuromancer.fetch("segment_id","neuron_name","version")
list(zip(ids,names,version))


[(96631955273149705, 'RONNY', 0),
 (103178515351946567, 'LEAH', 0),
 (103178515351946567, 'LEAH', 1),
 (104726695973782750, 'DARREN', 0),
 (106626583548494129, 'WILSON', 0),
 (106626583548494129, 'WILSON', 1),
 (107816118160698192, 'CHARMAINE', 0),
 (110778132960975016, 'LOUISE', 0)]

In [101]:
ids,names,version = m65.FromNeuromancer.fetch("segment_id","neuron_name","version")
for speicific_id, specific_name,specific_version in list(zip(ids,names,version)):
    #ones that work with all main pieces and subpieces: 107816118160698192, 104726695973782750, 106626583548494129 version 1, 107816118160698192, 110778132960975016
    if speicific_id != 110778132960975016:
        continue
    search_dict = dict(segment_id=speicific_id,version=specific_version)
    data_fetched = (minnie.Decimation & search_dict).fetch()
    print(f"{speicific_id}:{specific_name}:{specific_version} data available: {len(data_fetched)}")
    
    
    lookup_key = search_dict
    subsegments = (m65.FromNeuromancer.Subsegment & lookup_key).fetch(as_dict=True)
    total_subsemgnet_length = sum([k["n_vertices"] for k in subsegments])
    print("Subsemgnet Extra Length = " + str(total_subsemgnet_length))
    
    if len(data_fetched)> 0:
        main_mesh_length = data_fetched[0]["n_vertices"]
        print(f"Mian mesh Length = {main_mesh_length}")
        print("Total Length = " + str(total_subsemgnet_length + main_mesh_length))
    
    subsegment_list = [k["subsegment_id"] for k in subsegments]
    total_subsegment_counter = 0
    for sb in subsegment_list:
        sd = dict(segment_id=sb)
        total_subsegment_counter += len((minnie.Decimation & sd).fetch())
    
    print(f"     Total subsegments present = {total_subsegment_counter}/{len(subsegment_list)}")
    print(f"-------------------------------------------\n")

110778132960975016:LOUISE:0 data available: 1
Subsemgnet Extra Length = 4523643
Mian mesh Length = 435963
Total Length = 4959606
     Total subsegments present = 68/68
-------------------------------------------



In [ ]:
"""
Conclusion:
- only the 0 versions are available
- a lot of the mesh pieces aren't available
"""

In [102]:
"""
Ones that are in there:
107816118160698192 (CHARMAINE)
104726695973782750 (DARREN)

"""

search_dict = dict(segment_id=103178515351946567)
data_fetched = (minnie.Decimation & search_dict).fetch()
len(data_fetched)

1

In [103]:
lookup_key = dict(segment_id=107816118160698192,version = 0)
subsegments = (m65.FromNeuromancer.Subsegment & lookup_key).fetch(as_dict=True)
subsegment_list = [k["subsegment_id"] for k in subsegments]
subsegment_lengths = [k["n_vertices"] for k in subsegments]
total_extra_length = 0
total_counter = 0
segments_missed = []
for sb,sb_len in zip(subsegment_list,subsegment_lengths):
    search_dict = dict(segment_id=sb)
    current_result = len((minnie.Decimation & search_dict).fetch())
    total_counter += current_result
    print(f"Subsgment {sb} of length {sb_len} : {current_result}")
    total_extra_length += sb_len
    if current_result<=0:
        segments_missed.append(sb)
    
    #pull them down from the subsegments and print out the list
print("Total Counter =  + " + str(total_counter))
print("Total Extra Length = " + str(total_extra_length))

Subsgment 101902395407434514 of length 669 : 1
Subsgment 102887695332199785 of length 24515 : 1
Subsgment 102889619544428494 of length 31891 : 1
Subsgment 102890719257233540 of length 681 : 1
Subsgment 103029257454175365 of length 570 : 1
Subsgment 103029738490480020 of length 5888 : 1
Subsgment 103242493990490048 of length 252 : 1
Subsgment 103313137746848709 of length 3918 : 1
Subsgment 103665737381631891 of length 55495 : 1
Subsgment 103733425932104471 of length 105 : 1
Subsgment 103870246544622156 of length 5011 : 1
Subsgment 104508237898222004 of length 4395 : 1
Subsgment 104859394290245283 of length 13880 : 1
Subsgment 105144579648752618 of length 603 : 1
Subsgment 105846205908604595 of length 5061 : 1
Subsgment 105849023474493209 of length 1572 : 1
Subsgment 106060060920826917 of length 427 : 1
Subsgment 106130429664994368 of length 551 : 1
Subsgment 106130429665017025 of length 17 : 1
Subsgment 106341673403199619 of length 1217 : 1
Subsgment 106412042080209043 of length 552 : 1

In [ ]:
segments_missed

In [ ]:
"""
Conclusion: Confirmed that the mesh pieces weren't properly decimated for a specifi neuron
"""

# CHECKING THE ACUTAL SIZES OF THE MESH PIECES

# Creating the whole stitched together mesh

In [ ]:
import datetime

#combine all the meshes into one mesh
def add_mesh_piece(main_mesh_vertices,main_mesh_faces,sub_mesh_vertices,sub_mesh_faces):
    """
    Purpose: Takes in a large mesh piece and an array of other meshes and 
    returns a large mesh with all meshes appended
    
    Parameters:
    main_mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates
    main_mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices
    sub_mesh_vertices(list of np.arrays) : list of np arrays with the vertices arrays for all subsegments to be added
    sub_mesh_faces(list of np.arrays) : list of np arrays with the faces arrays for all subsegments to be added
    
    Returns:
    mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates for NEW CONCATENATED MESH
    mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices for NEW CONCATENATED MESH
    
    
    Pseudocode: 
    - Checks: 
    a. Make sure there sub_mesh arrays are greater than 0 and of the same length

    1) Count the number of vertices and faces in the main mesh
    2) Iterate through the submesh vertices and faces. In loop:
    a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
    b. Add the vertices_count and add that to every number in the faces array
    c. Concatenate the submesh faces onto the larger mesh face
    d. Save this new vertices and faces as the main_mesh verts and faces
    e. Print out how many new vertices and faces added
    3) Print out number of segments added, total faces/vertices for new mesh
    4) Return the main mesh vertices and faces
    
    """
    #a. Make sure there sub_mesh arrays are greater than 0 and of the same length
    if len(sub_mesh_vertices) <= 0:
        print("There were no vertices in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) <= 0:
        print("There were no face in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) != len(sub_mesh_vertices):
        raise Exception("The sub_mesh_faces and sub_mesh_vertices length did not match")
        
    
    #1) Count the number of vertices and faces in the main mesh
    n_main_vertices = len(main_mesh_vertices)
    n_main_faces = len(main_mesh_faces)
    
    
    #2) Iterate through the submesh vertices and faces. In loop:
    for i,(sub_verts, sub_faces) in enumerate(zip(sub_mesh_vertices,sub_mesh_faces)):
        #a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
        n_sub_verts = len(sub_verts)
        n_sub_faces = len(sub_faces)
        
        main_mesh_vertices = np.vstack([main_mesh_vertices,sub_verts])

        
        #b. Add the vertices_count of main to every number in the faces array
        sub_faces = sub_faces + n_main_vertices
        
        #c. Concatenate the submesh faces onto the larger mesh face
        main_mesh_faces = np.vstack([main_mesh_faces,sub_faces])
        
        #d. Save this new vertices and faces as the main_mesh verts and faces (DONE)
        
        #e. Print out how many new vertices and faces added
        #print(f"Added subsegment {i} with {n_sub_verts} vertices and {n_sub_faces} faces")
        
        n_main_vertices = len(main_mesh_vertices)
        n_main_faces = len(main_mesh_faces)
    
    #3) Print out number of segments added, total faces/vertices for new mesh  
    print(f"Added {len(sub_mesh_vertices)} subsegements \n  --> final mesh: {len(main_mesh_vertices)} vertices and {len(main_mesh_faces)} faces")
        
    return main_mesh_vertices,main_mesh_faces 

# @schema
# class WholeAutoAnnotationsLabelClusters3(dj.Computed):
#     definition="""
#     -> m65.FromNeuromancer
#     decimation_ratio     : decimal(3,2)                 # percentage that the faces are decimated
#     author_original      : varchar(20)                  # name of last editor
#     author_proofreader   : varchar(20)                  # name of last editor
#     date_time=CURRENT_TIMESTAMP : timestamp                    # the last time it was edited
#     ---
#     vertices             : longblob                     # label data for the vertices
#     triangles            : longblob                     # label data for the faces
#     edges                : longblob                     # label data for the edges
#     status               : varchar(16)                  # the index of the status descriptor that can be references by the StatusKey
#     """
    
#     def make(self,key):
#         """
#         Pseudocode: 
#         1) Download the main mesh and all its subsegments and Concatenate the meshes according to the correct order
#         3) Create blank labels for the size of the vertices, triangles and edges and make parameters:
#         a. decimation = 1
#         author_original = computer_Auto
#         author_proofreader = bcelii
#         date_time = current time
#         status = complete
#         4) Insert the key into the table
#         """
        
#         #1) Download the main mesh and all its subsegments and Concatenate the meshes according to the correct order
#         n = (m65.FromNeuromancer & key).fetch1()
        
#         print("---Main Neuron ---")
#         print(n["segment_id"])
#         print(n["n_vertices"])
#         print(n["vertices"])
#         print(n["faces"])

#         #get all of the segments and their data
#         lookup_key = dict(segment_id=n["segment_id"],version = n["version"])
#         subsegments = (m65.FromNeuromancer.Subsegment & lookup_key).fetch(as_dict=True)

#         subsegment_dicts = dict([(k["subsegment_id"],dict(vertices=k["vertices"],faces=k["faces"])) for k in subsegments])

#         subsegment_ordered_list = np.sort(np.array(list(subsegment_dicts.keys())))
#         subsegments_vertices = [subsegment_dicts[k]["vertices"] for k in subsegment_ordered_list]
#         subsegments_faces = [subsegment_dicts[k]["faces"] for k in subsegment_ordered_list]
        
        
#         new_mesh_vertices, new_mesh_faces = add_mesh_piece(main_mesh_vertices=n["vertices"],
#                    main_mesh_faces=n["faces"],
#                    sub_mesh_vertices = subsegments_vertices,
#                    sub_mesh_faces=subsegments_faces)
        
#         #3) Create blank labels for the size of the vertices, triangles and edges
        
#         insert_key = dict(key)
#         insert_key["vertices"] = np.zeros(len(new_mesh_vertices))
#         insert_key["edges"] = np.array([])
#         insert_key["triangles"] = np.zeros(len(new_mesh_faces))
#         insert_key["decimation_ratio"] = 1.0
#         insert_key["author_original"] = "computer_Auto"
#         insert_key["author_proofreader"] = "bcelii"
#         insert_key["status"] = "complete"
#         insert_key["date_time"] = date_time = str(datetime.datetime.now())[0:19]
        
#         #4) Insert the key into the table
#         self.insert1(insert_key,skip_duplicates=True)
        

In [ ]:
@schema
class WholeAutoAnnotationsLabelClusters3(dj.Computed):
    definition="""
    -> m65.FromNeuromancer
    decimation_ratio     : decimal(3,2)                 # percentage that the faces are decimated
    author_original      : varchar(20)                  # name of last editor
    author_proofreader   : varchar(20)                  # name of last editor
    date_time=CURRENT_TIMESTAMP : timestamp                    # the last time it was edited
    ---
    vertices             : longblob                     # label data for the vertices
    triangles            : longblob                     # label data for the faces
    edges                : longblob                     # label data for the edges
    status               : varchar(16)                  # the index of the status descriptor that can be references by the StatusKey
    """
    
    def make(self,key):
        """
        Pseudocode: 
        1) Download the main mesh and all its subsegments and Concatenate the meshes according to the correct order
        3) Create blank labels for the size of the vertices, triangles and edges and make parameters:
        a. decimation = 1
        author_original = computer_Auto
        author_proofreader = bcelii
        date_time = current time
        status = complete
        4) Insert the key into the table
        """
        
        #1) Download the main mesh and all its subsegments and Concatenate the meshes according to the correct order
        n = (m65.FromNeuromancer & key).fetch1()
        
        print("---Main Neuron ---")
        print(n["segment_id"])
        print(n["n_vertices"])
        print(n["vertices"])
        print(n["faces"])

        #get all of the segments and their data
        lookup_key = dict(segment_id=n["segment_id"],version = n["version"])
        subsegments = (m65.FromNeuromancer.Subsegment & lookup_key).fetch(as_dict=True)

        subsegment_dicts = dict([(k["subsegment_id"],dict(vertices=k["vertices"],faces=k["faces"])) for k in subsegments])

        subsegment_ordered_list = np.sort(np.array(list(subsegment_dicts.keys())))
        subsegments_vertices = [subsegment_dicts[k]["vertices"] for k in subsegment_ordered_list]
        subsegments_faces = [subsegment_dicts[k]["faces"] for k in subsegment_ordered_list]
        
        
        new_mesh_vertices, new_mesh_faces = add_mesh_piece(main_mesh_vertices=n["vertices"],
                   main_mesh_faces=n["faces"],
                   sub_mesh_vertices = subsegments_vertices,
                   sub_mesh_faces=subsegments_faces)
        
        #3) Create blank labels for the size of the vertices, triangles and edges
        
        insert_key = dict(key)
        insert_key["vertices"] = np.zeros(len(new_mesh_vertices))
        insert_key["edges"] = np.array([])
        insert_key["triangles"] = np.zeros(len(new_mesh_faces))
        insert_key["decimation_ratio"] = 1.0
        insert_key["author_original"] = "computer_Auto"
        insert_key["author_proofreader"] = "bcelii"
        insert_key["status"] = "complete"
        insert_key["date_time"] = date_time = str(datetime.datetime.now())[0:19]
        
        #4) Insert the key into the table
        self.insert1(insert_key,skip_duplicates=True)
        

In [ ]:
subsegments = (m65.FromNeuromancer.Subsegment & lookup_key).fetch(as_dict=True)
subsegment_list = [k["subsegment_id"] for k in subsegments]
subsegment_lengths = [k["n_vertices"] for k in subsegments]
total_extra_length = 0
total_counter = 0
segments_missed = []
for sb,sb_len in zip(subsegment_list,subsegment_lengths):
    search_dict = dict(segment_id=sb)
    current_result = len((minnie.Decimation & search_dict).fetch())
    total_counter += current_result
    print(f"Subsgment {sb} of length {sb_len} : {current_result}")
    total_extra_length += sb_len
    if current_result<=0:
        segments_missed.append(sb)

In [ ]:
vertices = data_fetched[0]["mesh"].vertices
faces = data_fetched[0]["mesh"].faces

In [ ]:
def find_highest_version(list_of_dicts,attr):
    attr_list = [k[attr] for k in list_of_dicts]
    return np.argmax(attr_list)

In [ ]:
subsegments = (m65.FromNeuromancer.Subsegment & lookup_key).fetch(as_dict=True)
subsegment_list = [k["subsegment_id"] for k in subsegments]
len(subsegment_list)len((minnie.Decimation & [dict(segment_id=k,version=decimation_version) for k in subsegment_list]))


In [104]:
decimation_version = 0
decimation_ratio = 0.25
def check_all_segments_decimated(key):
    subsegments = (m65.FromNeuromancer.Subsegment & key).fetch(as_dict=True)
    subsegment_list = [k["subsegment_id"] for k in subsegments]
    neuroglancer_list_len = len(subsegment_list)
    decimation_list_len = len((minnie.Decimation & [dict(segment_id=k,version=decimation_version,
                                                        decimation_ratio=decimation_ratio) for k in subsegment_list]))
    if decimation_list_len < neuroglancer_list_len:
        print(f"For {key['segment_id']} Only {neuroglancer_list_len} / {decimation_list_len} subsegments from the neuroglancer list are in the decimation table for decimation version {decimation_version}")
        return False
    else:
        print(f"{key['segment_id']} correctly had {neuroglancer_list_len} / {decimation_list_len} subsegments from the neuroglancer list and decimation table for decimation version {decimation_version}")
        return True

def fetch_stitched_mesh(key,return_list=True):
    """
    Purpose: Downloads the returns mesh data with the base mesh stitched to all its submeshes
    """
     
    #0) Check that all subsegments are decimated
    decimation_subsegments_check = check_all_segments_decimated(key)
    if not decimation_subsegments_check:
        return (np.array([]),np.array([]))
    
    #1) Download the main mesh and all its subsegments and Concatenate the meshes according to the correct order
    try:
        n = (minnie.Decimation & dict(segment_id=key["segment_id"],
                                      version=decimation_version,
                                     decimation_ratio=decimation_ratio)).fetch1()
    except:
        print(f"The main mesh {key['segment_id']} was not in the decimation table for decimation_version = {decimation_version}")
        return (np.array([]),np.array([]))
    
    """ THIS WOULD ENSURE WOULD ALWAYS USE THE HIGHEST VERSION 
    highest_dec_version = find_highest_version(data_fetched,"version")
    n = data_fetched[highest_dec_version]
    """

    print("---Main Neuron ---")
    print(n["segment_id"])
    print(n["n_vertices"])
    n["vertices"] = n["mesh"].vertices
    n["faces"] = n["mesh"].faces

    #get all of the segments and their data (THIS WILL ALREADY HAVE THE VERSION NUMBER IN THERE)
    subsemgnet_lookup_key = dict(segment_id=key["segment_id"],version = key["version"])
    subsegments = (m65.FromNeuromancer.Subsegment & subsemgnet_lookup_key).fetch(as_dict=True)

    subsegment_ids = [k["subsegment_id"] for k in subsegments]
    subsegments_decimated = (minnie.Decimation & [dict(segment_id=k,
                                      version=decimation_version,
                                     decimation_ratio=decimation_ratio)
                                    for k in subsegment_ids]).fetch(as_dict=True)

    subsegments_vertices = [k["mesh"].vertices for k in subsegments_decimated]
    subsegments_faces = [k["mesh"].faces for k in subsegments_decimated]

    new_mesh_vertices, new_mesh_faces = add_mesh_piece(main_mesh_vertices=n["vertices"],
       main_mesh_faces=n["faces"],
       sub_mesh_vertices = subsegments_vertices,
       sub_mesh_faces=subsegments_faces)
    if return_list == False:
        return new_mesh_vertices.astype(dtype=np.int32),new_mesh_faces.astype(dtype=np.int32)
    else:
        return new_mesh_vertices.astype(dtype=np.int32).tolist(),new_mesh_faces.astype(dtype=np.int32).tolist()

def fetch_stitched_mesh_statistics_vp2(key):
    """
    Purpose: Downloads the returns vertices and faces count for the base mesh stitched to all its submeshes
    To replace querying the PymeshfixDec.... table to check the vertices count
    """

    #1) Download the main mesh and all its subsegments and Concatenate the meshes according to the correct order
    main_n_vertices,main_n_faces = (minnie.Decimation & dict(segment_id=key["segment_id"],version=decimation_version)).fetch1("n_vertices","n_faces")

    #get all of the segments and their data
    
    #get all of the segments and their data (THIS WILL ALREADY HAVE THE VERSION NUMBER IN THERE)
    subsemgnet_lookup_key = dict(segment_id=key["segment_id"],version = key["version"])
    subsegments = (m65.FromNeuromancer.Subsegment & subsemgnet_lookup_key).fetch(as_dict=True)

    subsegment_ids = [k["subsegment_id"] for k in subsegments]
    subsegments_verts,subsegments_faces = (minnie.Decimation & [dict(segment_id=k,
                                      version=decimation_version,
                                     decimation_ratio=decimation_ratio)
                                    for k in subsegment_ids]).fetch("n_vertices","n_faces")
    
    
    return np.sum(subsegments_verts) + main_n_vertices,np.sum(subsegments_faces) + main_n_faces


lookup_key = dict(segment_id=107816118160698192,version = 0)
verts,faces = fetch_stitched_mesh(lookup_key,return_list=False)
verts_num,faces_num =  fetch_stitched_mesh_statistics_vp2(lookup_key)
print(verts_num,faces_num)

107816118160698192 correctly had 46 / 46 subsegments from the neuroglancer list and decimation table for decimation version 0
---Main Neuron ---
107816118160698192
224209
Added 46 subsegements 
  --> final mesh: 299150 vertices and 597515 faces
299150 597515


In [105]:
# output the mesh
import trimesh
new_mesh = trimesh.Trimesh(verts,faces)

In [107]:
new_mesh.export("example_minnie_neuron.off")
print("hello")

hello


In [ ]:
"""
VERSIONS OF FUNCTIONS THAT WILL WORK IN BLENDER
"""

decimation_version = 0
decimation_ratio = 0.25
def check_all_segments_decimated(key):
    subsegments = (m65.FromNeuromancer.Subsegment & key).fetch(as_dict=True)
    subsegment_list = [k["subsegment_id"] for k in subsegments]
    neuroglancer_list_len = len(subsegment_list)
    decimation_list_len = len((minnie.Decimation & [dict(segment_id=k,version=decimation_version,
                                                        decimation_ratio=decimation_ratio) for k in subsegment_list]))
    if decimation_list_len < neuroglancer_list_len:
        raise Excpetion("The number of subsegments in decimation table was less than needed")
        return False
    else:
        return True

def fetch_stitched_mesh(key,return_list=True):
    """
    Purpose: Downloads the returns mesh data with the base mesh stitched to all its submeshes
    """
     
    #0) Check that all subsegments are decimated
    decimation_subsegments_check = check_all_segments_decimated(key)
    if not decimation_subsegments_check:
        raise Exception("Error occured in the decimation_subsegments_check")
        return (np.array([]),np.array([]))
    
    #1) Download the main mesh and all its subsegments and Concatenate the meshes according to the correct order
    try:
        n = (minnie.Decimation & dict(segment_id=key["segment_id"],
                                      version=decimation_version,
                                     decimation_ratio=decimation_ratio)).fetch1()
    except:
        raise Exception("The main mesh was not able to be retrieved from decimation table")
        return (np.array([]),np.array([]))
    
    """ THIS WOULD ENSURE WOULD ALWAYS USE THE HIGHEST VERSION 
    highest_dec_version = find_highest_version(data_fetched,"version")
    n = data_fetched[highest_dec_version]
    """

    print("---Main Neuron ---")
    print(n["segment_id"])
    print(n["n_vertices"])
    n["vertices"] = n["mesh"].vertices
    n["faces"] = n["mesh"].faces

    #get all of the segments and their data (THIS WILL ALREADY HAVE THE VERSION NUMBER IN THERE)
    subsemgnet_lookup_key = dict(segment_id=key["segment_id"],version = key["version"])
    subsegments = (m65.FromNeuromancer.Subsegment & subsemgnet_lookup_key).fetch(as_dict=True)

    subsegment_ids = [k["subsegment_id"] for k in subsegments]
    subsegments_decimated = (minnie.Decimation & [dict(segment_id=k,
                                      version=decimation_version,
                                     decimation_ratio=decimation_ratio)
                                    for k in subsegment_ids]).fetch(as_dict=True)

    subsegments_vertices = [k["mesh"].vertices for k in subsegments_decimated]
    subsegments_faces = [k["mesh"].faces for k in subsegments_decimated]

    new_mesh_vertices, new_mesh_faces = add_mesh_piece(main_mesh_vertices=n["vertices"],
       main_mesh_faces=n["faces"],
       sub_mesh_vertices = subsegments_vertices,
       sub_mesh_faces=subsegments_faces)
    if return_list == False:
        return new_mesh_vertices.astype(dtype=np.int32),new_mesh_faces.astype(dtype=np.int32)
    else:
        return new_mesh_vertices.astype(dtype=np.int32).tolist(),new_mesh_faces.astype(dtype=np.int32).tolist()

def fetch_stitched_mesh_statistics_vp2(key):
    """
    Purpose: Downloads the returns vertices and faces count for the base mesh stitched to all its submeshes
    To replace querying the PymeshfixDec.... table to check the vertices count
    """

    #1) Download the main mesh and all its subsegments and Concatenate the meshes according to the correct order
    main_n_vertices,main_n_faces = (minnie.Decimation & dict(segment_id=key["segment_id"],version=decimation_version)).fetch1("n_vertices","n_faces")

    #get all of the segments and their data
    
    #get all of the segments and their data (THIS WILL ALREADY HAVE THE VERSION NUMBER IN THERE)
    subsemgnet_lookup_key = dict(segment_id=key["segment_id"],version = key["version"])
    subsegments = (m65.FromNeuromancer.Subsegment & subsemgnet_lookup_key).fetch(as_dict=True)

    subsegment_ids = [k["subsegment_id"] for k in subsegments]
    subsegments_verts,subsegments_faces = (minnie.Decimation & [dict(segment_id=k,
                                      version=decimation_version,
                                     decimation_ratio=decimation_ratio)
                                    for k in subsegment_ids]).fetch("n_vertices","n_faces")
    
    
    return np.sum(subsegments_verts) + main_n_vertices,np.sum(subsegments_faces) + main_n_faces


lookup_key = dict(segment_id=107816118160698192,version = 0)
verts,faces = fetch_stitched_mesh(lookup_key,return_list=False)
verts_num,faces_num =  fetch_stitched_mesh_statistics_vp2(lookup_key)
print(verts_num,faces_num)

# Making the decimation tables: 

In [ ]:
m65 = dj.create_virtual_module('m65', 'microns_minnie65_01')
schema = dj.schema("microns_minnie65_01")

In [ ]:
#making the table for the Exclusion Criteria
@schema
class ExclusionCriteria(dj.Manual):
    definition="""
    criteria_id          : int unsigned                 # key by which to lookup the exclusion criteria description
    ---
    criteria_desc        : varchar(64)                  # criteria by which this segment was excluded (preferred to add name of excluder as well)
    """
    
#ExclusionCriteria.insert1(dict(criteria_id=0,criteria_desc="Blender Error Labeling"))

In [ ]:
#making the table for the SynapseExclude
@schema
class SynapseExclude(dj.Manual):
    definition="""
    -> m65.Synapse
    version              : tinyint unsigned             # version of proofreading for the neuron
    -> m65.ExclusionCriteria
    ---
    -> m65.Segment.proj(presyn="segment_id")
    -> m65.Segment.proj(postsyn="segment_id")
    """
SynapseExclude()

In [ ]:
ExclusionCriteria()

In [ ]:
pinky.Authors.describe()
print("hello")

@schema
class Authors(dj.Manual):
    definition="""
    # maps numeric labels to descriptive labels
    username             : varchar(20)                  # username the person pcisk
    ---
    real_name            : varchar(40)                  # the real name of that corresponds to the username
    """
    a
Authors.insert([dict(username="celiib",real_name="Brendan Celii"),dict(username="spapadop",real_name="stelios papadopoulos")])

# creat the new LabelKey
total_Label_data = pinky.LabelKey.fetch()

@schema
class LabelKey(dj.Manual):
    definition="""
    # maps numeric labels to descriptive and color labels
    numeric              : tinyint unsigned             # numeric label of the compartment
    ---
    description          : varchar(16)                  # descriptive name of the label
    color                : varchar(16)                  # representative color of the compartment
    blender_colors       : tinyblob                     # color RGB values to be used in blender
    """
    
LabelKey.insert(total_Label_data)

# The tables that Blender reads from and writes to

In [ ]:
m65.WholeAutoAnnotationsLabelClusters3.delete()

In [ ]:
m65.ProofreadLabel.delete()

In [ ]:
# test to see if an array is empty
x,y = np.array([]),np.array([])
x.size

In [ ]:
@schema
class ProofreadLabel(dj.Manual):
    definition="""
    -> m65.FromNeuromancer
    decimation_ratio     : decimal(3,2)                 # percentage that the faces are decimated
    author_original      : varchar(20)                  # name of last editor
    author_proofreader   : varchar(20)                  # name of last editor
    date_time=CURRENT_TIMESTAMP : timestamp                    # the last time it was edited
    ---
    vertices             : longblob                     # label data for the vertices
    triangles            : longblob                     # label data for the faces
    edges                : longblob                     # label data for the edges
    status               : varchar(16)                  # the index of the status descriptor that can be references by the StatusKey
    """

# Table that will propogate the decimated versions

In [110]:
"""
These versions of the code roll through if have errors in fething the decimation
"""

decimation_version = 0
decimation_ratio = 0.25
def check_all_segments_decimated(key):
    subsegments = (m65.FromNeuromancer.Subsegment & key).fetch(as_dict=True)
    subsegment_list = [k["subsegment_id"] for k in subsegments]
    neuroglancer_list_len = len(subsegment_list)
    decimation_list_len = len((minnie.Decimation & [dict(segment_id=k,version=decimation_version,
                                                        decimation_ratio=decimation_ratio) for k in subsegment_list]))
    if decimation_list_len < neuroglancer_list_len:
        print(f"For {key['segment_id']} Only {neuroglancer_list_len} / {decimation_list_len} subsegments from the neuroglancer list are in the decimation table for decimation version {decimation_version}")
        return False
    else:
        print(f"{key['segment_id']} correctly had {neuroglancer_list_len} / {decimation_list_len} subsegments from the neuroglancer list and decimation table for decimation version {decimation_version}")
        return True

def fetch_stitched_mesh(key,return_list=True):
    """
    Purpose: Downloads the returns mesh data with the base mesh stitched to all its submeshes
    """
     
    #0) Check that all subsegments are decimated
    decimation_subsegments_check = check_all_segments_decimated(key)
    if not decimation_subsegments_check:
        return (np.array([]),np.array([]))
    
    #1) Download the main mesh and all its subsegments and Concatenate the meshes according to the correct order
    try:
        n = (minnie.Decimation & dict(segment_id=key["segment_id"],
                                      version=decimation_version,
                                     decimation_ratio=decimation_ratio)).fetch1()
    except:
        print(f"The main mesh {key['segment_id']} was not in the decimation table for decimation_version = {decimation_version}")
        return (np.array([]),np.array([]))
    
    """ THIS WOULD ENSURE WOULD ALWAYS USE THE HIGHEST VERSION 
    highest_dec_version = find_highest_version(data_fetched,"version")
    n = data_fetched[highest_dec_version]
    """

    print("---Main Neuron ---")
    print(n["segment_id"])
    print(n["n_vertices"])
    n["vertices"] = n["mesh"].vertices
    n["faces"] = n["mesh"].faces

    #get all of the segments and their data (THIS WILL ALREADY HAVE THE VERSION NUMBER IN THERE)
    subsemgnet_lookup_key = dict(segment_id=key["segment_id"],version = key["version"])
    subsegments = (m65.FromNeuromancer.Subsegment & subsemgnet_lookup_key).fetch(as_dict=True)

    subsegment_ids = [k["subsegment_id"] for k in subsegments]
    try:
        subsegments_decimated = (minnie.Decimation & [dict(segment_id=k,
                                          version=decimation_version,
                                         decimation_ratio=decimation_ratio)
                                        for k in subsegment_ids]).fetch(as_dict=True)
    except:
        print(f"There was a corrupted subsegment_id {key['segment_id']}  for decimation_version = {decimation_version}")
        return (np.array([]),np.array([]))
        

    subsegments_vertices = [k["mesh"].vertices for k in subsegments_decimated]
    subsegments_faces = [k["mesh"].faces for k in subsegments_decimated]

    new_mesh_vertices, new_mesh_faces = add_mesh_piece(main_mesh_vertices=n["vertices"],
       main_mesh_faces=n["faces"],
       sub_mesh_vertices = subsegments_vertices,
       sub_mesh_faces=subsegments_faces)
    if return_list == False:
        return new_mesh_vertices.astype(dtype=np.int32),new_mesh_faces.astype(dtype=np.int32)
    else:
        return new_mesh_vertices.astype(dtype=np.int32).tolist(),new_mesh_faces.astype(dtype=np.int32).tolist()

@schema
class WholeAutoAnnotationsLabelClusters3(dj.Computed):
    definition="""
    -> m65.FromNeuromancer
    decimation_ratio     : decimal(3,2)                 # percentage that the faces are decimated
    author_original      : varchar(20)                  # name of last editor
    author_proofreader   : varchar(20)                  # name of last editor
    date_time=CURRENT_TIMESTAMP : timestamp                    # the last time it was edited
    ---
    vertices             : longblob                     # label data for the vertices
    triangles            : longblob                     # label data for the faces
    edges                : longblob                     # label data for the edges
    status               : varchar(16)                  # the index of the status descriptor that can be references by the StatusKey
    """
    
    def make(self,key):
        """
        Pseudocode: 
        1) Download the main mesh and all its subsegments and Concatenate the meshes according to the correct order
        3) Create blank labels for the size of the vertices, triangles and edges and make parameters:
        a. decimation = 1
        author_original = computer_Auto
        author_proofreader = bcelii
        date_time = current time
        status = complete
        4) Insert the key into the table
        """
        print(f"----Working on {key['segment_id']}--------")
        #3) Create blank labels for the size of the vertices, triangles and edges
        verts,faces = returned_mesh = fetch_stitched_mesh(key,return_list=False)
        if verts.size <= 0 or faces.size <=0:
            print("There was an error with generating the mesh")
            return

        
        insert_key = dict(key)
        insert_key["vertices"] = np.zeros(len(verts))
        insert_key["edges"] = np.array([])
        insert_key["triangles"] = np.zeros(len(faces))
        insert_key["decimation_ratio"] = decimation_ratio
        insert_key["author_original"] = "computer_Auto"
        insert_key["author_proofreader"] = "bcelii"
        insert_key["status"] = "complete"
        insert_key["date_time"] = date_time = str(datetime.datetime.now())[0:19]
        
        #4) Insert the key into the table
        self.insert1(insert_key,skip_duplicates=True)

In [111]:
#(schema.jobs & 'table_name="__whole_auto_annotations_label_clusters3"').delete()

In [112]:
#(schema.jobs & "table_name='__whole_auto_annotations_label_clusters3'")#.delete()
dj.config["enable_python_native_blobs"] = True

import time
start_time = time.time()
WholeAutoAnnotationsLabelClusters3.populate(reserve_jobs=True)
print(f"Total time for WholeAutoAnnotationsLabelClusters3 populate = {time.time() - start_time}")

----Working on 96631955273149705--------
96631955273149705 correctly had 80 / 80 subsegments from the neuroglancer list and decimation table for decimation version 0
---Main Neuron ---
96631955273149705
395000
Added 80 subsegements 
  --> final mesh: 443295 vertices and 886554 faces
----Working on 103178515351946567--------
103178515351946567 correctly had 75 / 75 subsegments from the neuroglancer list and decimation table for decimation version 0
---Main Neuron ---
103178515351946567
1541976
Added 75 subsegements 
  --> final mesh: 1713548 vertices and 3449010 faces
----Working on 103178515351946567--------
103178515351946567 correctly had 80 / 80 subsegments from the neuroglancer list and decimation table for decimation version 0
---Main Neuron ---
103178515351946567
1541976
Added 80 subsegements 
  --> final mesh: 1750023 vertices and 3522088 faces
----Working on 106626583548494129--------
106626583548494129 correctly had 69 / 69 subsegments from the neuroglancer list and decimation

In [113]:
m65.WholeAutoAnnotationsLabelClusters3()

"segment_id segment id unique within each Segmentation, should be mirrored in Segment table.",version version of proofreading for the neuron,decimation_ratio percentage that the faces are decimated,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices label data for the vertices,triangles label data for the faces,edges label data for the edges,status the index of the status descriptor that can be references by the StatusKey
96631955273149705,0,0.25,computer_Auto,bcelii,2020-02-04 16:07:08,=BLOB=,=BLOB=,=BLOB=,complete
103178515351946567,0,0.25,computer_Auto,bcelii,2020-02-04 16:07:14,=BLOB=,=BLOB=,=BLOB=,complete
103178515351946567,1,0.25,computer_Auto,bcelii,2020-02-04 16:07:19,=BLOB=,=BLOB=,=BLOB=,complete
104726695973782750,0,0.25,computer_Auto,bcelii,2020-01-29 17:10:08,=BLOB=,=BLOB=,=BLOB=,complete
106626583548494129,0,0.25,computer_Auto,bcelii,2020-02-04 16:07:23,=BLOB=,=BLOB=,=BLOB=,complete
106626583548494129,1,0.25,computer_Auto,bcelii,2020-01-29 17:10:11,=BLOB=,=BLOB=,=BLOB=,complete
107816118160698192,0,0.25,computer_Auto,bcelii,2020-01-29 17:10:12,=BLOB=,=BLOB=,=BLOB=,complete
110778132960975016,0,0.25,computer_Auto,bcelii,2020-01-29 17:10:19,=BLOB=,=BLOB=,=BLOB=,complete


In [94]:
current_version=0
current_decimation=0.25
labels = m65.ProofreadLabel()
primary_key = dict(version=current_version,decimation_ratio=current_decimation) 
mesh_list = (m65.WholeAutoAnnotationsLabelClusters3  & primary_key).fetch("segment_id").tolist() 
#mesh_list = mesh_Table.fetch("segment_id").tolist() #old way of doing it without decimated list
labels_list = labels.fetch("segment_id").tolist()

difference = [x for x in mesh_list if x not in labels_list]
#print(difference)

In [96]:
m65.WholeAutoAnnotationsLabelClusters3()

"segment_id segment id unique within each Segmentation, should be mirrored in Segment table.",version version of proofreading for the neuron,decimation_ratio percentage that the faces are decimated,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices label data for the vertices,triangles label data for the faces,edges label data for the edges,status the index of the status descriptor that can be references by the StatusKey
104726695973782750,0,0.25,computer_Auto,bcelii,2020-01-29 17:10:08,=BLOB=,=BLOB=,=BLOB=,complete
106626583548494129,1,0.25,computer_Auto,bcelii,2020-01-29 17:10:11,=BLOB=,=BLOB=,=BLOB=,complete
107816118160698192,0,0.25,computer_Auto,bcelii,2020-01-29 17:10:12,=BLOB=,=BLOB=,=BLOB=,complete
110778132960975016,0,0.25,computer_Auto,bcelii,2020-01-29 17:10:19,=BLOB=,=BLOB=,=BLOB=,complete
